## Tree based methods in Spark

Sushant N. More

Ref: Udemy course - Python and Spark for Big data.

In [1]:
import findspark

In [2]:
findspark.init('/home/sushant/spark-2.1.0-bin-hadoop2.7')

In [3]:
from pyspark.sql import SparkSession

In [4]:
spark = SparkSession.builder.appName('funTree').getOrCreate()

In [13]:
from pyspark.ml import Pipeline
from pyspark.ml.classification import RandomForestClassifier, GBTClassifier, DecisionTreeClassifier

Looking at the documentation examples in MLlib to begin with.

In [8]:
data = spark.read.format('libsvm').load('./Python-and-Spark-for-Big-Data-master/Spark_for_Machine_Learning/Tree_Methods/sample_libsvm_data.txt', 
                                       inferSchema=True, header=True)

In [11]:
data.show(5)

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  0.0|(692,[127,128,129...|
|  1.0|(692,[158,159,160...|
|  1.0|(692,[124,125,126...|
|  1.0|(692,[152,153,154...|
|  1.0|(692,[151,152,153...|
+-----+--------------------+
only showing top 5 rows



In [10]:
data.printSchema()

root
 |-- label: double (nullable = true)
 |-- features: vector (nullable = true)



In [15]:
train, test = data.randomSplit([0.7, 0.3])

In [16]:
dtc = DecisionTreeClassifier()
rfc = RandomForestClassifier()
gbt = GBTClassifier()

In [17]:
dtc_model = dtc.fit(train)
rfc_model = rfc.fit(train)
gbt_model = gbt.fit(train)

Now predict for the test data

In [18]:
dtc_preds = dtc_model.transform(test)
rfc_preds = rfc_model.transform(test)
gbt_preds = gbt_model.transform(test)

In [28]:
test.show(2)

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  0.0|(692,[122,123,124...|
|  0.0|(692,[123,124,125...|
+-----+--------------------+
only showing top 2 rows



In [19]:
dtc_preds.show(4)

+-----+--------------------+-------------+-----------+----------+
|label|            features|rawPrediction|probability|prediction|
+-----+--------------------+-------------+-----------+----------+
|  0.0|(692,[122,123,124...|   [31.0,0.0]|  [1.0,0.0]|       0.0|
|  0.0|(692,[123,124,125...|   [31.0,0.0]|  [1.0,0.0]|       0.0|
|  0.0|(692,[124,125,126...|   [31.0,0.0]|  [1.0,0.0]|       0.0|
|  0.0|(692,[124,125,126...|   [31.0,0.0]|  [1.0,0.0]|       0.0|
+-----+--------------------+-------------+-----------+----------+
only showing top 4 rows



In [20]:
rfc_preds.show(4)

+-----+--------------------+-------------+-----------+----------+
|label|            features|rawPrediction|probability|prediction|
+-----+--------------------+-------------+-----------+----------+
|  0.0|(692,[122,123,124...|   [20.0,0.0]|  [1.0,0.0]|       0.0|
|  0.0|(692,[123,124,125...|   [20.0,0.0]|  [1.0,0.0]|       0.0|
|  0.0|(692,[124,125,126...|   [20.0,0.0]|  [1.0,0.0]|       0.0|
|  0.0|(692,[124,125,126...|   [20.0,0.0]|  [1.0,0.0]|       0.0|
+-----+--------------------+-------------+-----------+----------+
only showing top 4 rows



In [21]:
gbt_preds.show(4)

+-----+--------------------+----------+
|label|            features|prediction|
+-----+--------------------+----------+
|  0.0|(692,[122,123,124...|       0.0|
|  0.0|(692,[123,124,125...|       0.0|
|  0.0|(692,[124,125,126...|       0.0|
|  0.0|(692,[124,125,126...|       0.0|
+-----+--------------------+----------+
only showing top 4 rows



Note that GBT does not have the rawPrediction column. BinaryClassClassificationEvaluator and multiclassClassification Evaluator have rawPrediction as the default input value. 

In [22]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
# Even though this is binary classification we use the MulticlassClassificationEvaluator. 
# This allows us to get metrics like precision, accuracy directly. BinaryclassClassificationEvaluator 
# has only two metrics auc under roc and area under precision-recall curve.

In [23]:
acc_eval = MulticlassClassificationEvaluator(metricName='accuracy')

In [24]:
acc_eval.evaluate(dtc_preds)

0.9629629629629629

In [25]:
acc_eval.evaluate(rfc_preds)

0.9629629629629629

In [26]:
acc_eval.evaluate(gbt_preds)

0.9629629629629629

In [27]:
rfc_model.featureImportances

SparseVector(692, {243: 0.0028, 267: 0.0027, 291: 0.0032, 320: 0.0028, 327: 0.0074, 345: 0.0389, 359: 0.0234, 370: 0.0073, 375: 0.0185, 377: 0.0461, 385: 0.0374, 405: 0.05, 407: 0.0447, 411: 0.0053, 412: 0.0122, 416: 0.0037, 429: 0.004, 433: 0.0124, 435: 0.0475, 455: 0.0546, 456: 0.0112, 461: 0.1419, 462: 0.0497, 463: 0.0051, 466: 0.0022, 467: 0.0081, 483: 0.0426, 490: 0.1025, 497: 0.0311, 516: 0.0034, 517: 0.0975, 523: 0.0325, 540: 0.04, 544: 0.0054, 626: 0.0019})

### Predicting if the college is public or private depending on the features

In [31]:
collegeData = spark.read.csv('./Python-and-Spark-for-Big-Data-master/Spark_for_Machine_Learning/Tree_Methods/College.csv',
                             inferSchema=True, header=True)

In [32]:
collegeData.printSchema()

root
 |-- School: string (nullable = true)
 |-- Private: string (nullable = true)
 |-- Apps: integer (nullable = true)
 |-- Accept: integer (nullable = true)
 |-- Enroll: integer (nullable = true)
 |-- Top10perc: integer (nullable = true)
 |-- Top25perc: integer (nullable = true)
 |-- F_Undergrad: integer (nullable = true)
 |-- P_Undergrad: integer (nullable = true)
 |-- Outstate: integer (nullable = true)
 |-- Room_Board: integer (nullable = true)
 |-- Books: integer (nullable = true)
 |-- Personal: integer (nullable = true)
 |-- PhD: integer (nullable = true)
 |-- Terminal: integer (nullable = true)
 |-- S_F_Ratio: double (nullable = true)
 |-- perc_alumni: integer (nullable = true)
 |-- Expend: integer (nullable = true)
 |-- Grad_Rate: integer (nullable = true)



In [34]:
collegeData.show(2)

+--------------------+-------+----+------+------+---------+---------+-----------+-----------+--------+----------+-----+--------+---+--------+---------+-----------+------+---------+
|              School|Private|Apps|Accept|Enroll|Top10perc|Top25perc|F_Undergrad|P_Undergrad|Outstate|Room_Board|Books|Personal|PhD|Terminal|S_F_Ratio|perc_alumni|Expend|Grad_Rate|
+--------------------+-------+----+------+------+---------+---------+-----------+-----------+--------+----------+-----+--------+---+--------+---------+-----------+------+---------+
|Abilene Christian...|    Yes|1660|  1232|   721|       23|       52|       2885|        537|    7440|      3300|  450|    2200| 70|      78|     18.1|         12|  7041|       60|
|  Adelphi University|    Yes|2186|  1924|   512|       16|       29|       2683|       1227|   12280|      6450|  750|    1500| 29|      30|     12.2|         16| 10527|       56|
+--------------------+-------+----+------+------+---------+---------+-----------+-----------+--

In [35]:
collegeData.head(2)

[Row(School='Abilene Christian University', Private='Yes', Apps=1660, Accept=1232, Enroll=721, Top10perc=23, Top25perc=52, F_Undergrad=2885, P_Undergrad=537, Outstate=7440, Room_Board=3300, Books=450, Personal=2200, PhD=70, Terminal=78, S_F_Ratio=18.1, perc_alumni=12, Expend=7041, Grad_Rate=60),
 Row(School='Adelphi University', Private='Yes', Apps=2186, Accept=1924, Enroll=512, Top10perc=16, Top25perc=29, F_Undergrad=2683, P_Undergrad=1227, Outstate=12280, Room_Board=6450, Books=750, Personal=1500, PhD=29, Terminal=30, S_F_Ratio=12.2, perc_alumni=16, Expend=10527, Grad_Rate=56)]

In [36]:
from pyspark.ml.feature import VectorAssembler

In [37]:
collegeData.columns

['School',
 'Private',
 'Apps',
 'Accept',
 'Enroll',
 'Top10perc',
 'Top25perc',
 'F_Undergrad',
 'P_Undergrad',
 'Outstate',
 'Room_Board',
 'Books',
 'Personal',
 'PhD',
 'Terminal',
 'S_F_Ratio',
 'perc_alumni',
 'Expend',
 'Grad_Rate']

In [38]:
assembler = VectorAssembler(inputCols=['Apps',
 'Accept',
 'Enroll',
 'Top10perc',
 'Top25perc',
 'F_Undergrad',
 'P_Undergrad',
 'Outstate',
 'Room_Board',
 'Books',
 'Personal',
 'PhD',
 'Terminal',
 'S_F_Ratio',
 'perc_alumni',
 'Expend',
 'Grad_Rate'], outputCol='features')

In [39]:
output = assembler.transform(collegeData)